In [1]:
# -*- coding: utf-8 -*-
import arcpy
import requests
import json
import pandas as pd

In [21]:
url = 'https://restapi.amap.com/v3/direction/transit/integrated?key={}&\
origin={}&destination={}&city=%E8%A5%BF%E5%AE%89&cityd=%E8%A5%BF%E5%AE%89&strategy=0&nightflag=0'.format('50ace0067201128f9781da8128e942f0','109.121117,34.447047','108.883543,34.197582')
print(url)

https://restapi.amap.com/v3/direction/transit/integrated?key=50ace0067201128f9781da8128e942f0&origin=109.121117,34.447047&destination=108.883543,34.197582&city=%E8%A5%BF%E5%AE%89&cityd=%E8%A5%BF%E5%AE%89&strategy=0&nightflag=0


In [2]:
dest = pd.read_excel(u'D:/rui/sw_analysis/homework/part5.comprehensive/chapter3/城市实时交通出行数据_数据模板/输入_终点_WGS84坐标系.xlsx')
dest.head()

,did,d_lng,d_lat,city
0,1,108.754458,34.116286,西安
1,2,108.759875,34.116385,西安
2,3,108.765293,34.116483,西安
3,4,108.770710,34.116582,西安
4,5,108.776128,34.116680,西安


In [15]:
import sys
sys.path.append(r'D:\rui\code_analysis\homework\Project\coordTransform_py')
import coordTransform_utils

In [18]:
coordTransform_utils.wgs84_to_gcj02(108.754458,34.116286)

[108.75917088895646, 34.11479662189801]

In [23]:
coord_trans_url = 'https://restapi.amap.com/v3/assistant/coordinate/convert?key=50ace0067201128f9781da8128e942f0&locations=108.754458,34.116286&coordsys=gps'
print(coord_trans_url)

https://restapi.amap.com/v3/assistant/coordinate/convert?key=50ace0067201128f9781da8128e942f0&locations=108.754458,34.116286&coordsys=gps


In [25]:
res = requests.get(coord_trans_url)

In [28]:
content = res.text
print(content)

{"status":"1","info":"ok","infocode":"10000","locations":"108.759189181858,34.114811740452"}


In [33]:
content_json = json.loads(content)
content_json['locations']

u'108.759189181858,34.114811740452'

In [3]:
def to_gcj02(key,locations,coordsys):
    url = 'https://restapi.amap.com/v3/assistant/coordinate/convert\
?key={}&locations={}&coordsys={}'.format(key,locations,coordsys)
    res = requests.get(url).text
    content = json.loads(res)
    return content['locations']
    
coordsys = 'gps'
key = '50ace0067201128f9781da8128e942f0'
locations = '108.754458,34.116286|108.759875,34.116385'

test = to_gcj02(key,locations,coordsys)
print(test)

108.759189181858,34.114811740452;108.764607204862,34.114909396702


In [61]:
test.split(';') + test.split(';')

[u'108.759189181858,34.114811740452',
 u'108.764607204862,34.114909396702',
 u'108.759189181858,34.114811740452',
 u'108.764607204862,34.114909396702']

In [5]:
dest['locations'] = dest.d_lng.map(str) + ',' + dest.d_lat.map(str)

In [46]:
dest.head()

,did,d_lng,d_lat,city,locations
0,1,108.754458,34.116286,西安,"108.754457504,34.1162855455"
1,2,108.759875,34.116385,西安,"108.759875051,34.1163845254"
2,3,108.765293,34.116483,西安,"108.765292626,34.1164832666"
3,4,108.770710,34.116582,西安,"108.770710227,34.1165817691"
4,5,108.776128,34.116680,西安,"108.776127854,34.1166800328"


In [50]:
len(dest.locations)

5110

In [51]:
len(dest.locations)/40

127

In [53]:
len(dest.locations)%40

30

In [58]:
("|").join(dest.locations[40*126:40*127])

'108.745809812,34.4407008605|108.75124818,34.4408010449|108.756686576,34.4409009877|108.762124998,34.4410006889|108.767563446,34.4411001484|108.773001921,34.4411993664|108.778440423,34.4412983426|108.783878952,34.4413970773|108.789317506,34.4414955703|108.794756087,34.4415938217|108.800194695,34.4416918314|108.805633328,34.4417895994|108.811071988,34.4418871258|108.816510674,34.4419844106|108.821949386,34.4420814537|108.827388124,34.4421782551|108.832826888,34.4422748148|108.838265677,34.4423711329|108.843704493,34.4424672093|108.849143334,34.442563044|108.854582201,34.442658637|108.860021093,34.4427539883|108.865460011,34.4428490979|108.870898955,34.4429439659|108.876337923,34.4430385921|108.881776918,34.4431329766|108.887215937,34.4432271195|108.892654982,34.4433210206|108.898094051,34.44341468|108.903533146,34.4435080976|108.908972266,34.4436012736|108.914411411,34.4436942078|108.91985058,34.4437869003|108.925289775,34.443879351|108.930728994,34.44397156|108.936168238,34.4440635273|

In [24]:
def get_locations():
    num = len(dest.locations)/40
    last = len(dest.locations)%40
    lsts = []
    for i in range(num):
        locations = ("|").join(dest.locations[40*i:40*(i+1)])
        try:
            lst = to_gcj02(key,locations,coordsys).split(';')
            lsts = lsts + lst 
        except:
            print('try again!')
            lst = to_gcj02(key,locations,coordsys).split(';')
            lsts = lsts + lst     
    last_lac = ("|").join(dest.locations[-last:])
    last_lst = to_gcj02(key,last_lac,coordsys).split(';')
    lsts = lsts + last_lst
    return lsts

In [ ]:
dest['gcj02_locations'] = get_locations()

In [26]:
key = '50ace0067201128f9781da8128e942f0'
origin = '109.121117,34.447047'
destination = '108.883543,34.197582'
def get_traffic(key,origin,destination):
    url = 'https://restapi.amap.com/v3/direction/transit/integrated?key={}&\
origin={}&destination={}&city=%E8%A5%BF%E5%AE%89&cityd=%E8%A5%BF%E5%AE%89&strategy=0&nightflag=0'.format(key,origin,destination)
    res = requests.get(url).text.encode('utf8')
    content = json.loads(res)
    return content

In [27]:
test3 = get_traffic(key,origin,destination)

In [41]:
print test3['route']['transits'][0]['segments'][0]['walking']['steps'][0]['instruction']

步行44米左转


In [42]:
print test3['route']['taxi_cost']

141.891


In [ ]:
print test3['route']['taxi_cost']